In [2]:
import os, sys

%reload_ext autoreload
%autoreload 2

# DATASET

Collecting the dataset from the remote host&nbsp;:

In [66]:
from my_TS_Anomaly_lib import download_nasa_ims_bearings_dataset

root_dir = os.path.join( '.', 'data' )

download_nasa_ims_bearings_dataset(
    root_dir
)

     100%|██████████████████████████████████████████████████| 262530/262530 [03:18<00:00, 1322.22it/s]
Extracting 4 entries :
	['1st_test.rar', '2nd_test.rar', '3rd_test.rar', 'Readme Document for IMS Bearing Data.pdf'] done [00:01:37].
Extracting sub-folder #1/3 (1st_test) :
     100%|██████████████████████████████████████████████████| 2157/2157 [03:40<00:00,  9.78it/s]
Extracting sub-folder #2/3 (2nd_test) :
     100%|██████████████████████████████████████████████████| 985/985 [01:17<00:00, 12.76it/s]
Extracting sub-folder #3/3 (3rd_test) :
     100%|██████████████████████████████████████████████████| 6326/6326 [07:25<00:00, 14.20it/s]


completed in 00:17:20


# MONGODB

In [ ]:
# Launch the mongoDB local server instance (from command shell on Mongo DB server)
# "C:\Program Files\MongoDB\Server\4.4\bin\mongod.exe" --dbpath="D:\mongodb_data_dir" --bind_ip 127.0.0.1 --auth

Get a connection from Python Jupyter Notebook&nbsp;:

In [2]:
from pymongo import MongoClient
from pprint import pprint

username = 'myAdmin'
password = 'my_admin_password'

client = MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password)
                     , appname='NASA IMS')

Import the source files to a new database on the Mongo DB server&nbsp;:

In [6]:
from my_TS_Anomaly_lib import measurements_to_mongo
from multiprocessing import cpu_count

root_dir =  os.path.join( '.', 'data' )
database_name = 'nasa_ims_database'

measurements_to_mongo( client, database_name, root_dir
                       , par_backend = 'multiprocessing', n_jobs = cpu_count()-1 )

if not client.admin['system.users'].find_one({'user': 'readOnlyUser'}) :
    client.admin.command(
        "createUser", 'readOnlyUser', pwd='readOnlyUser_password'
        , roles=[{'role': 'read','db': database_name}]
    )


Transferring data to Mongo DB :



Imported 194 Million 'timestep' documents


<p style="font-family: monospace">
Transferring data to Mongo DB :</p>

<div>
<img style="vertical-align:middle;" width="310px" align="left" alt="measurements to mongo widget" src="./images/tqdm_widget.png?uncache=464453868" />

<ul style="line-height: 34px; list-style-type: none;">
<li style="margin: 0px 10px 0px 30px;">
<p style="font-family: Helvetica; font-size: 10pt;">9464/9464 [26:27&lt;00:00, 5.96it/s]</p>
</li>
</ul>
</div>

<p style="font-family: monospace">
Imported 194 Million 'timestep' documents.</p>

As part of the <b><em>measurements_to_mongo</em></b> convenience method, an index on the 'test_id' attribute has been created&nbsp;:

In [8]:
client[database_name].measurements.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'test_id_idx': {'v': 2, 'key': [('test_id', -1)], 'background': True}}

<hr style="height:2px;border-width:0;color:gray;background-color:gray;width:80%" />

Let's look into one measurement timestep document for each of the 3 bearings test-to-failure experiments&nbsp;:

In [14]:
from datetime import datetime

cursor = client.nasa_ims_database.measurements.aggregate([
    # group by test_id,
    # keep first document of each group,
    # get all attributes for each returned document
    {'$group': {
        '_id': "$test_id"
        , 'test_Document': {'$first': {
            "$objectToArray": "$$ROOT"
        }}
    }},
    # sort result by increasing test_id
    {'$sort': {"_id": 1}},
    # convert from {'k': xxx, 'v': yyyy}
    # to {xxx: yyyy} format
    {"$replaceRoot": {
        "newRoot": {
            "$arrayToObject": "$test_Document"
        }
    }},
    # remove the '_id' attribute from the returned document
    {"$project": {"_id": 0}}
])

while cursor._has_next() :
    cursor_document = cursor.next()
    print("example of measurement document for test #" +
          str(cursor_document['test_id']) + " :")
    pprint({x: cursor_document[x]
            for x in cursor_document if x not in ['timestamp']})
    print("'timestamp': " + str(cursor_document['timestamp']))
    print()


example of measurement document for test #1 :
{'sensor_1': '-0.022',
 'sensor_2': '-0.039',
 'sensor_3': '-0.183',
 'sensor_4': '-0.054',
 'sensor_5': '-0.105',
 'sensor_6': '-0.134',
 'sensor_7': '-0.129',
 'sensor_8': '-0.142',
 'test_id': 1}
'timestamp': 2003-10-22 12:06:24

example of measurement document for test #2 :
{'sensor_1': '-0.049',
 'sensor_2': '-0.071',
 'sensor_3': '-0.132',
 'sensor_4': '-0.010',
 'test_id': 2}
'timestamp': 2004-02-12 10:32:39

example of measurement document for test #3 :
{'sensor_1': '0.034',
 'sensor_2': '0.264',
 'sensor_3': '0.039',
 'sensor_4': '-0.046',
 'test_id': 3}
'timestamp': 2004-03-04 09:27:46



Notice above that test 1 was performed with 8 sensors (i.e. 2 per bearing) whereas both test 2 & 3 were covered with only 4 sensors (i.e. 1 per bearing).

<hr style="height:2px;border-width:0;color:gray;background-color:gray;width:80%" />

In [67]:
# list all databases on the Mongo DB server :
client.list_database_names()

['admin', 'config', 'local', 'nasa_ims_database']

In [37]:
# list all collections in the (newly created) database :
client.nasa_ims_database.list_collection_names()

['measurements']

In [5]:
# DEBUG
# client.nasa_ims_database.command("dropDatabase")

{'dropped': 'nasa_ims_database', 'ok': 1.0}

<hr style="height:2px;border-width:0;color:gray;background-color:gray;width:80%" />

Count all documents attributes (from measurements, all timesteps, all sensors). This operation requires a complete database collection scan&nbsp;:

In [28]:
from my_TS_Anomaly_lib.utils import millify

batch_size = 70_000

# we use the 'aggregate' operator to apply projection query
# (exclusion projection, addFields, inclusion projection)
cursor = client.nasa_ims_database['measurements'].aggregate([
    # exclusion projection (do not consider these attributes in what will follow)
    { "$project": {
        "_id": 0
        , "timestamp": 0
        , "test_id": 0
    } },
    # addFields 'count' : count the attributes of each returned records
    { "$addFields": {
        "count": {
            "$size": { 
                "$objectToArray": "$$ROOT"
            }
        }
    } },
    # inclusion projection, only keep the 'count' (added) attribute
    { "$project": {
        "count": 1
    } }
    #, { "$limit": 10 }
]
).batch_size(batch_size)



from tqdm.notebook import tqdm as notebook_tqdm

print("Counting the sensors for all timesteps :")
attributes_counter = 0 ; document_counter = 0 ; pbar_rate = 10_000
total = client.nasa_ims_database.measurements \
            .estimated_document_count()
with notebook_tqdm(
    total = total
) as pbar :
    for document in cursor :
        document_counter += 1
        attributes_counter += document['count']
        if document_counter % pbar_rate == 0 : pbar.update(pbar_rate)
    pbar.n = total ; pbar.refresh()

print( millify(attributes_counter) + " sensory measurements in total" )


952 Million sensory measurements in total


<p style="font-family: monospace">
Counting the sensors for all timesteps&nbsp;:</p>

<div>
<img style="vertical-align:middle;" width="310px" align="left" alt="measurements to mongo widget" src="./images/tqdm_widget.png?uncache=464453868" />

<ul style="line-height: 34px; list-style-type: none;">
<li style="margin: 0px 10px 0px 30px;">
<p style="font-family: Helvetica; font-size: 10pt;">193822720/193822720 [26:04&lt;00:00, 123902.03it/s]
</li>
</ul>
</div>

<p style="font-family: monospace">
952 Million sensory measurements in total</p>

<br />
<br />
<br />
<center><b><font size=14pt>THE END.</font></b></center>
<br />
<br />
<br />

<hr style="height:2px;border-width:0;color:gray;background-color:gray;width:80%" />

# EXTRA

<b>Export Notebook to HTML (with Markdown extension cells evaluated and hidden cell outputs omitted)&nbsp;:</b>

In [3]:
from my_TS_Anomaly_lib.jupyter_markdown_extension import md_extension_to_html

md_extension_to_html(os.path.join(os.path.realpath('.'), 'measurements_to_mongo.ipynb'))

'measurements_to_mongo.ipynb' ; 'D:\jupyter_notebooks\TimeSeries_Anomaly_Detection\measurements_to_mongo.html'
done.


<hr style="height: 4px; width: 70%; margin:0 auto;" />

<b>Updates to the user 'readOnlyUser'&nbsp;:</b>

In [156]:
# list all users :
pprint(client.admin.command('usersInfo'))

{'ok': 1.0,
 'users': [{'_id': 'admin.myAdmin',
            'db': 'admin',
            'mechanisms': ['SCRAM-SHA-1', 'SCRAM-SHA-256'],
            'roles': [{'db': 'admin', 'role': 'root'}],
            'user': 'myAdmin',
            'userId': UUID('56ad1f34-b558-4970-90b6-3b85bab5a2b0')},
           {'_id': 'admin.myUserAdmin',
            'db': 'admin',
            'mechanisms': ['SCRAM-SHA-1', 'SCRAM-SHA-256'],
            'roles': [{'db': 'admin', 'role': 'userAdminAnyDatabase'},
                      {'db': 'admin', 'role': 'readWriteAnyDatabase'}],
            'user': 'myUserAdmin',
            'userId': UUID('b3c75ece-c910-4257-a44f-63f05e8f2e96')},
           {'_id': 'admin.readOnlyUser',
            'db': 'admin',
            'mechanisms': ['SCRAM-SHA-1', 'SCRAM-SHA-256'],
            'roles': [{'db': 'nasa_ims_database', 'role': 'read'}],
            'user': 'readOnlyUser',
            'userId': UUID('a27cdd77-a790-4e78-a262-047225a0be99')}]}


In [32]:
# drop 'readOnlyUser'
client.admin.command('dropUser', 'readOnlyUser')

{'ok': 1.0}

In [33]:
# create 'readOnlyUser'
client.admin.command("createUser", 'readOnlyUser', pwd='readOnlyUser_password'
    , roles=[{'role':'read','db':'nasa_ims_database'}]
)

{'ok': 1.0}

In [34]:
print(client.admin.list_collection_names())
# list role/database pairs
pprint(
    client.admin['system.users'].find_one(
        {'user': 'readOnlyUser'}
        , ['roles']
    )['roles']
)

['system.users', 'system.version']
[{'db': 'nasa_ims_database', 'role': 'read'}]


In [30]:
# add 'read' role on 'test' db to 'readOnlyUser' =>
client.admin.command("updateUser"
    , "readOnlyUser"
    , roles = client.admin['system.users'].find_one(
                    {'user': 'readOnlyUser'}
                    , ['roles']
                )['roles'] +
                [{'role':'read','db':'test'}]
)

{'ok': 1.0}

<hr style="height: 4px; width: 70%; margin:0 auto;" />